In [ ]:
# !pip3 install folium
# !pip3 install requests
# !pip3 install hvplot
# !pip3 install pandas



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python3.10 -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python3.10 -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python3.10 -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [1]:
import folium 
from io import BytesIO
import requests
import subprocess
import hvplot.pandas
import pandas as pd

hv.extension('matplotlib')
hv.extension('bokeh')
warnings.filterwarnings('ignore')

%opts magic unavailable (pyparsing cannot be imported)
%compositor magic unavailable (pyparsing cannot be imported)


NameError: name 'hv' is not defined

## STEP 2: SITE MAP AND DESCRIPTION
<!-- ![](kern_river.mov) -->


### All throughout the Sierra drainage rivers flooded and reservoirs refilled
This analysis focuses on the Kern River, near Onyx CA.

### Site Description
  * The data was collected at South Fork Kern River in Onyx, CA, site code 11189500
  * The region is characterized with rugged mountains and a high desert climate. The majority of precipitation feeding the river arrives as snowfall between December and March.
  * The agency that manages and collects this data is USGS
  * You can access this usgs tat through their [national water dashboard](https://dashboard.waterdata.usgs.gov/app/nwd/en/?aoi=default)
 

### Site Map: The Kern River near Onyx, CA

In [ ]:
sg_lat = 35.7374516
sg_lon = -118.173689

# Initialize map and tweak settings
m = folium.Map(
    # Location to display
    location=(sg_lat, sg_lon),
    # Turns off annoying zooming while trying to scroll to the next cell
    scrollWheelZoom=False)

# Put a marker at the stream gauge location
folium.Marker([sg_lat, sg_lon], popup="Thingy").add_to(m)

# Display the map
m

&#9998; USGS streamflow URL: *url here*

## Data Exploration

In [ ]:
streamflow_url = ("https://waterdata.usgs.gov/nwis/dv?cb_00060=on&format"
                  "=rdb&site_no=11189500&legacy=&referred_module=sw&period"
                  "=&begin_date=1911-09-12&end_date=2023-09-22"
    )

# streamflow_url

'https://waterdata.usgs.gov/nwis/dv?cb_00060=on&format=rdb&site_no=11189500&legacy=&referred_module=sw&period=&begin_date=1911-09-12&end_date=2023-09-22'

In [ ]:
# Send an HTTP GET request to the URL
nwis_response = requests.get(streamflow_url)

# Check if the request was successful (status code 200)
if nwis_response.status_code == 200:
    # Print the content of the nwis_response (HTML, JSON, etc.)
    print(nwis_response.text)
else:
    print(f"Request failed with status code {nwis_response.status_code}")

# ---------------------------------- WARNING ----------------------------------------
# Some of the data that you have obtained from this U.S. Geological Survey database
# may not have received Director's approval. Any such data values are qualified
# as provisional and are subject to revision. Provisional data are released on the
# condition that neither the USGS nor the United States Government may be held liable
# for any damages resulting from its use.
#
# Additional info: https://help.waterdata.usgs.gov/policies/provisional-data-statement
#
# File-format description:  https://help.waterdata.usgs.gov/faq/about-tab-delimited-output
# Automated-retrieval info: https://help.waterdata.usgs.gov/faq/automated-retrievals
#
# Contact:   gs-w_waterdata_support@usgs.gov
# retrieved: 2023-09-23 17:12:18 EDT       (vaww01)
#
# Data for the following 1 site(s) are contained in this file
#    USGS 11189500 SF KERN R NR ONYX CA
# -------------------------------------------------------------------

In [ ]:
# Print the top of the data
for i, line in enumerate(nwis_response.content.splitlines()[:10]):
    print(i, line)

0 b'# ---------------------------------- WARNING ----------------------------------------'
1 b'# Some of the data that you have obtained from this U.S. Geological Survey database'
2 b"# may not have received Director's approval. Any such data values are qualified"
3 b'# as provisional and are subject to revision. Provisional data are released on the'
4 b'# condition that neither the USGS nor the United States Government may be held liable'
5 b'# for any damages resulting from its use.'
6 b'#'
7 b'# Additional info: https://help.waterdata.usgs.gov/policies/provisional-data-statement'
8 b'#'
9 b'# File-format description:  https://help.waterdata.usgs.gov/faq/about-tab-delimited-output'


In [ ]:
# Take a look at the data. What got downloaded?
for i, line in enumerate(nwis_response.content.splitlines()[:35]):
    # Skip commented lines
    if not line.startswith(b'#'):
        print(i, line)

27 b'agency_cd\tsite_no\tdatetime\t43245_00060_00003\t43245_00060_00003_cd'
28 b'5s\t15s\t20d\t14n\t10s'
29 b'USGS\t06610000\t1928-10-01\t20300\tA'
30 b'USGS\t06610000\t1928-10-02\t19800\tA'
31 b'USGS\t06610000\t1928-10-03\t19600\tA'
32 b'USGS\t06610000\t1928-10-04\t19300\tA'
33 b'USGS\t06610000\t1928-10-05\t19000\tA'
34 b'USGS\t06610000\t1928-10-06\t18000\tA'


In [ ]:
# Import the data using pandas

discharge_df = pd.read_csv(
    BytesIO(nwis_response.content),
    comment='#',
    delimiter='\t', 
    skiprows=[i for i in range(32)],
    names=['agency_code', 'site_number', 'date', 'stream_discharge_cfs', 'site_code'],
    index_col='date',
    parse_dates=['date'],
)

discharge_df

,agency_code,site_number,stream_discharge_cfs,site_code
date,,,,
1911-09-12,USGS,11189500,34.0,A
1911-09-13,USGS,11189500,34.0,A
1911-09-14,USGS,11189500,32.0,A
1911-09-15,USGS,11189500,31.0,A
1911-09-16,USGS,11189500,35.0,A
...,...,...,...,...
2023-09-18,USGS,11189500,74.1,P
2023-09-19,USGS,11189500,72.7,P
2023-09-20,USGS,11189500,71.4,P


### Data Schema

In [ ]:
discharge_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 40919 entries, 1911-09-12 to 2023-09-22
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   agency_code           40919 non-null  object 
 1   site_number           40919 non-null  int64  
 2   stream_discharge_cfs  35404 non-null  float64
 3   site_code             35404 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 1.6+ MB


### Data coverage & date range

In [ ]:
discharge_df.index

DatetimeIndex(['1911-09-12', '1911-09-13', '1911-09-14', '1911-09-15',
               '1911-09-16', '1911-09-17', '1911-09-18', '1911-09-19',
               '1911-09-20', '1911-09-21',
               ...
               '2023-09-13', '2023-09-14', '2023-09-15', '2023-09-16',
               '2023-09-17', '2023-09-18', '2023-09-19', '2023-09-20',
               '2023-09-21', '2023-09-22'],
              dtype='datetime64[ns]', name='date', length=40919, freq=None)

### Descriptive Statistics

In [ ]:
discharge_df.describe()

,site_number,stream_discharge_cfs
count,40919.0,35404.000000
mean,11189500.0,124.642727
std,0.0,289.311053
min,11189500.0,0.000000
25%,11189500.0,17.300000
50%,11189500.0,38.250000
75%,11189500.0,96.000000
max,11189500.0,14000.000000


## Data Visualizaton
I hiked the South Fork PCT, Kern river in May 2023. Flood waters were especially high at that time. The plots below highlight spring 2023 discharge in comparison to annual maxiumums from the last 110 years.

### Data sample Spring 2023

In [ ]:
spring_2023_df = discharge_df['2023-02-01':'2023-06-30']


spring_2023_df

,agency_code,site_number,stream_discharge_cfs,site_code
date,,,,
2023-02-01,USGS,11189500,55.7,P
2023-02-02,USGS,11189500,55.6,P
2023-02-03,USGS,11189500,55.1,P
2023-02-04,USGS,11189500,54.7,P
2023-02-05,USGS,11189500,61.4,P
...,...,...,...,...
2023-06-26,USGS,11189500,705.0,P
2023-06-27,USGS,11189500,664.0,P
2023-06-28,USGS,11189500,599.0,P


#### - Discharge Plot - Spring 2023

In [ ]:
discharge_plot_spring_2023 = spring_2023_df.hvplot(
    y='stream_discharge_cfs'
)

discharge_plot_spring_2023

/usr/local/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
/usr/local/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type


:Curve   [date]   (stream_discharge_cfs)

### Data plot - 1911 to 2023
All available data is represented in the plot below

In [ ]:
discharge_plot_all = discharge_df.hvplot(
    y='stream_discharge_cfs'
)

discharge_plot_all

/usr/local/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
/usr/local/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type


:Curve   [date]   (stream_discharge_cfs)

### Resample the data to display annual maximums only
#### Data sample of annual maximums 1911 to 2023

In [ ]:
discharge_annual_max_df = discharge_df[['stream_discharge_cfs']].resample(
    'AS').max()

discharge_annual_max_df

,stream_discharge_cfs
date,
1911-01-01,153.0
1912-01-01,327.0
1913-01-01,375.0
1914-01-01,NaN
1915-01-01,NaN
...,...
2019-01-01,1880.0
2020-01-01,327.0
2021-01-01,122.0


#### Plot of annual maximum discharge 1911 to 
The plot shows exceptionally high discharge in 1966 and 2023. Both these years had exceptionally high snowfall. California also experienced high snowfall in 1938 but that is not reflected in the stream discharge data from Onyx.

In [ ]:
discharge_annual_max_df.hvplot()

/usr/local/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
/usr/local/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type


:Curve   [date]   (stream_discharge_cfs)

In [ ]:
%%capture
%%bash
jupyter nbconvert time-series-data-kern-river.ipynb --to html --no-input